<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Adaboost" data-toc-modified-id="Adaboost-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Adaboost</a></span><ul class="toc-item"><li><span><a href="#Algorithm" data-toc-modified-id="Algorithm-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Algorithm</a></span></li><li><span><a href="#Voting" data-toc-modified-id="Voting-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Voting</a></span></li><li><span><a href="#Code" data-toc-modified-id="Code-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Code</a></span><ul class="toc-item"><li><span><a href="#Hyperparameters" data-toc-modified-id="Hyperparameters-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Hyperparameters</a></span></li></ul></li></ul></li><li><span><a href="#Gradient-Boosting" data-toc-modified-id="Gradient-Boosting-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Gradient Boosting</a></span><ul class="toc-item"><li><span><a href="#Algorithm" data-toc-modified-id="Algorithm-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Algorithm</a></span></li><li><span><a href="#Code" data-toc-modified-id="Code-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Code</a></span></li></ul></li></ul></div>

# Adaboost

## Algorithm

- Train model
- inflate errors
    + Errors increase weight to make it a 50-50
- Retrain model using errors (repeat)

![](images/adaboost.png)

## Voting

Combine weak learners

+ Truthful, Guesser, Liar ==> 1,0,-1 weights
+ $y= ln(\frac{x}{1-x})$ where $x$ is accuracy
+ $y=ln(\frac{correct}{incorrect})$ to log-odds
+ Vote by combining these calculated $y$ for each crossed-area (negative for "not blue" or whatever

## Code

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html

```python
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier()
model.fit(x_train, y_train)
model.predict(x_test)
```

### Hyperparameters

```python
from sklearn.tree import DecisionTreeClassifier
model = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth=4), n_estimators = 5)
```

`base_estimator`: The model utilized for the weak learners (Warning: Don't forget to import the model that you decide to use for the weak learner).
`n_estimators`: The maximum number of weak learners used.

# Gradient Boosting

Use gradient descent to improve the model

![](images/gradient_boosting_residuals.png)

## Algorithm

Use mean squared error (MSE) and want to minimize that <-- done by gradient descent

Use the residuals (pattern in the residuals) to create an even better model

1. Fit a model to the data, $F_1(x) = y$
2. Fit a model to the residuals, $h_1(x) = y - F_1(x)$
3. Create a new model, $F_2(x) = F_1(x) + h_1(x)$
4. Repeat


## Code

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html